In [1]:
import numpy as np
import glob
import glob
import os.path
import sys
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
import matplotlib.pyplot as plt
import imutils
import pandas as pd
import re
import datetime
from sklearn.decomposition import NMF
from sklearn.utils import shuffle
from skimage import draw
from skimage.io import imread
from scipy.sparse import csr_matrix,linalg
#import spams
import time
import math
from keras.models import Sequential,model_from_json, Input,Model
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,Dropout,ReLU,Activation
from keras.preprocessing import image
import keras.backend as K
from keras.utils import to_categorical
import random
from sklearn.metrics import f1_score
from tqdm import tqdm_notebook as tqdm 
#from tensorflow.keras.callbacks import TensorBoard
import keras
from tensorboard import version
from keras.models import model_from_json

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction=0.2
config.gpu_options.allow_growth=True ##to use gpu as needed
config.log_device_placement = True  
sess = tf.Session(config=config)
set_session(sess)




/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
!nvidia-smi

Wed Aug 14 15:15:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3D:00.0 Off |                    0 |
| N/A   36C    P0    58W / 300W |    934MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
print("Keras : {}".format(keras.__version__))
print("tensorflow : {}".format(tf.__version__))
print("tensorboard : {}".format(version.VERSION))

Keras : 2.2.4
tensorflow : 1.14.0
tensorboard : 1.14.0


In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

W0814 15:15:57.625965 140399403984704 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0814 15:15:57.628886 140399403984704 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
images_name=glob.glob('/datalab/nuclei_seg/Tissue_images/*.tif')
if not os.path.exists('/datalab/nuclei_seg/png_images'):
    os.mkdir('/datalab/nuclei_seg/png_images')
    print("made dir /datalab/nuclei_seg/png_images")
for c,im in tqdm(enumerate(images_name)):
    img_name=im
    img=cv2.imread(im,1)
    
    img_name=img_name.replace('Tissue_images','png_images')
    img_name=img_name.replace('.tif','.PNG')    
    cv2.imwrite(img_name,img)
    
   

In [ ]:
#!rm -rf /datalab/nuclei_seg/4_class/binary_images

In [5]:
new_name=glob.glob('/datalab/nuclei_seg/png_images/*.PNG')
BORDER_VALUE=255
NUCLEUS_VALUE=127
BG_VALUE=0


if not os.path.exists('/datalab/nuclei_seg/4_class/binary_images'):
    os.mkdir('/datalab/nuclei_seg/4_class/binary_images')
    print("made dir /datalab/nuclei_seg/4_class/binary_images")
    
def rot_image(img):
    test=img.copy()
    rot=imutils.rotate(test,270)
    rot=cv2.flip(rot,1)
    return rot
    
def poly2boundry(x,y,img_array):
    if len(x)==2 and len(y)==2:
        rr,cc=draw.line(x[0],y[0],x[1],y[1])
    else:
        rr, cc = draw.polygon_perimeter(x, y)
    img_array[rr,cc]=[BORDER_VALUE]
    return img_array

def check_in_bounds(x,y,bound):
    
    if x>=bound:
        x=bound-1
    if y>=bound:
        y=bound-1
    if x<0:
        x=0
    if y<0:
        y=0
    return x,y
    

for count,name in enumerate(new_name):
    print("File {} is {}".format(count,name))
    new_name=name.replace('/png_images','/4_class/binary_images')
    xml_name=name.replace('/datalab/nuclei_seg/png_images/','/datalab/nuclei_seg/Annotations/')
    xml_name=xml_name.replace('.PNG','.xml')
    tree=ET.parse(xml_name)
    root=tree.getroot()
    img_test=np.zeros(shape=(1000,1000),dtype=np.uint8)
    
    print("The number of regions :{}".format(len([v.tag for v in root.iter('Vertices')])))
    for v in root.iter('Vertices'):
        X=[]
        Y=[]

        for child in v:
            x=int(eval(child.attrib['X']))
            y=int(eval(child.attrib['Y']))
            x,y=check_in_bounds(x,y,1000)



            X.append(x)
            Y.append(y)


        img_temp=np.zeros(shape=(1000,1000),dtype=np.uint8)
        r_nucleus,c_nucleus=draw.polygon(X,Y)
        img_test[r_nucleus,c_nucleus]=NUCLEUS_VALUE
        img_temp[r_nucleus,c_nucleus]=NUCLEUS_VALUE

        img_test=poly2boundry(X,Y,img_test)

        area_nuc=len(r_nucleus)
        area_eroded=area_nuc
        kernel=np.array([np.array([0,1,0]),np.array([1,1,1]),np.array([0,1,0])],np.uint8)


        while area_eroded>0.4*area_nuc:
            area_eroded=len(np.where(img_temp==NUCLEUS_VALUE)[0])
            img_temp=cv2.erode(img_temp,kernel,iterations=1)
        
        '''
        for i,a in enumerate(x):
            img_test[a-1:a+2,y[i]-1:y[i]+2]=BORDER_VALUE
            img_test.dtype='uint8'
        '''
            
        img_test[np.where(img_temp==NUCLEUS_VALUE)]=63
    
    img_test.dtype=np.uint8
    x,y=np.where(img_test==255)
    for i,a in enumerate(x):
        img_test[a-1:a+2,y[i]-1:y[i]+2]=BORDER_VALUE
    img_test.dtype='uint8'
    
    cv2.imwrite(new_name,img_test)


    

    
    img_test=rot_image(img_test)
    cv2.imwrite(new_name,img_test)
    
print("DONE")




File 0 is /datalab/nuclei_seg/png_images/TCGA-G9-6348-01Z-00-DX1.PNG
The number of regions :390
File 1 is /datalab/nuclei_seg/png_images/TCGA-A7-A13F-01Z-00-DX1.PNG
The number of regions :356
File 2 is /datalab/nuclei_seg/png_images/TCGA-HE-7129-01Z-00-DX1.PNG
The number of regions :1585
File 3 is /datalab/nuclei_seg/png_images/TCGA-AR-A1AS-01Z-00-DX1.PNG
The number of regions :405
File 4 is /datalab/nuclei_seg/png_images/TCGA-HE-7130-01Z-00-DX1.PNG
The number of regions :1863
File 5 is /datalab/nuclei_seg/png_images/TCGA-B0-5710-01Z-00-DX1.PNG
The number of regions :359
File 6 is /datalab/nuclei_seg/png_images/TCGA-G9-6356-01Z-00-DX1.PNG
The number of regions :442
File 7 is /datalab/nuclei_seg/png_images/TCGA-18-5592-01Z-00-DX1.PNG
The number of regions :480
File 8 is /datalab/nuclei_seg/png_images/TCGA-E2-A1B5-01Z-00-DX1.PNG
The number of regions :329
File 9 is /datalab/nuclei_seg/png_images/TCGA-49-4488-01Z-00-DX1.PNG
The number of regions :557
File 10 is /datalab/nuclei_seg/png_ima

In [ ]:
def snnmf(I):
    
    I_lab_space = cv2.cvtColor(I, cv2.COLOR_RGB2LAB)
    luminosity = I_lab_space[:, :, 0] 
    mask_lumin = luminosity/ 255 < 0.8
    
    mask_zero = (I == 0)
    I[mask_zero] = 1
    
    od = (np.maximum(-1 * np.log(I / 255), 1e-6))
    od_dl = od[mask_lumin]
    od=od.reshape((-1, 3))
    od_dl=od_dl.reshape((-1, 3))

    dictionary = spams.trainDL(X=od_dl.T, K=2, lambda1=0.1, mode=2,
                               modeD=0, posAlpha=True, posD=True, verbose=False).T

    # dictionary is 2 x 3
    #First one should be Hematoxylin(Bluish-purple)
    if dictionary[0, 0] < dictionary[1, 0]:
        dictionary = dictionary[[1, 0], :]

    dictionary=dictionary / np.linalg.norm(dictionary, axis=1)[:, None]
    
    sparse=spams.lasso(X=od.T, D=dictionary.T, mode=2, lambda1=0.01, pos=True).toarray().T
    
    return dictionary,sparse

def h_norm(hs,ht):
    
    ht_rm=np.percentile(ht, 99, axis=0).reshape(1,2)
    hs_rm=np.percentile(hs, 99, axis=0).reshape(1,2)
    
    hs_norm_mat=hs*ht_rm/hs_rm
    return hs_norm_mat

def save_h(h,w,name,image_dims):
    
    for i in range(w.shape[1]):

        image_OD=255*np.exp(-np.dot(w[:,i].reshape((-1,1)),h[i,:].reshape((1,-1))))
        
        image_r=image_OD[0,:].reshape((1000,1000))
        image_g=image_OD[1,:].reshape((1000,1000))
        image_b=image_OD[2,:].reshape((1000,1000))
        
        image=cv2.merge((image_r,image_g,image_b))
        image=image.astype(np.uint8)
        
        bgr_image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        gray_image=cv2.cvtColor(bgr_image,cv2.COLOR_BGR2GRAY)
        
        cv2.imwrite('/datalab/nuclei_seg/{}stain_{}.PNG'.format(name,i+1),bgr_image)
        cv2.imwrite('/datalab/nuclei_seg/{}stain_{}_gray.PNG'.format(name,i+1),gray_image)

def pad_image(image):
    
    top,left,right,bottom=[25,25,25,25]
    BLACK = [0, 0, 0]
    
    constant_img = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT,value=[0,0,0])
    '''
    other parameters include
    cv2.BORDER_WRAP
    cv2.BORDER_REFLECT
    cv2.BORDER_REFLECT_101
    cv2.BORDER_CONSTANT,const_val
    cv2.BORDER_REPLICATE
    
    '''
    resized_image = cv2.resize(constant_img, (image.shape[0]+50, image.shape[1]+50))
    return resized_image
    
    
    

In [ ]:

imgt=cv2.imread('/datalab/nuclei_seg/png_images/TCGA-G9-6356-01Z-00-DX1.PNG',1)
pattern_norm=re.compile('/datalab/nuclei_seg/png_images/(.*).PNG')

if not os.path.exists('/datalab/nuclei_seg/norm'):
    os.mkdir('/datalab/nuclei_seg/norm')
    print(' norm directory has been made')

norm_images=glob.glob('/datalab/nuclei_seg/png_images/*.PNG')

wt,ht=snnmf(imgt)

for i_n,p_image in (enumerate(norm_images)):
    norm_file=pattern_norm.search(p_image) 
    norm_file=norm_file.group(1)

    
    imgs=cv2.imread(p_image,1)
    
      
    ws,hs=snnmf(imgs)

    hs_norm=h_norm(hs,ht)
    
    od_recon=255 * np.exp(-1 * np.dot(hs_norm, wt))

    norm_bgr_image=od_recon.reshape(imgt.shape).astype(np.uint8)
    cv2.imwrite('/datalab/nuclei_seg/norm/{}.PNG'.format(norm_file),norm_bgr_image)




In [ ]:

pattern=re.compile('/datalab/nuclei_seg/norm_ideal/(.*).PNG')
norm_images_name=glob.glob('/datalab/nuclei_seg/norm_ideal/*.PNG')
train_set=['TCGA-A7-A13E-01Z-00-DX1.PNG','TCGA-A7-A13F-01Z-00-DX1.PNG',\
                  'TCGA-AR-A1AK-01Z-00-DX1.PNG','TCGA-AR-A1AS-01Z-00-DX1.PNG',\
                  'TCGA-B0-5711-01Z-00-DX1.PNG','TCGA-HE-7128-01Z-00-DX1.PNG',\
                  'TCGA-HE-7129-01Z-00-DX1.PNG','TCGA-HE-7130-01Z-00-DX1.PNG',\
                  'TCGA-18-5592-01Z-00-DX1.PNG','TCGA-38-6178-01Z-00-DX1.PNG',\
                  'TCGA-49-4488-01Z-00-DX1.PNG','TCGA-50-5931-01Z-00-DX1.PNG',\
                  'TCGA-G9-6336-01Z-00-DX1.PNG','TCGA-G9-6348-01Z-00-DX1.PNG',\
                  'TCGA-G9-6363-01Z-00-DX1.PNG','TCGA-CH-5767-01Z-00-DX1.PNG']
images_name_train=random.sample(train_set,12)


if not os.path.exists('/datalab/nuclei_seg/4_class/train'):
    os.mkdir('/datalab/nuclei_seg/4_class/train')
    print(' train directory has been made')
if not os.path.exists('/datalab/nuclei_seg/4_class/train/nucleus'):
    os.mkdir('/datalab/nuclei_seg/4_class/train/nucleus')
    print("/datalab/nuclei_seg/4_class/train/nucleus dir has been made")
if not os.path.exists('/datalab/nuclei_seg/4_class/train/border'):
    os.mkdir('/datalab/nuclei_seg/4_class/train/border')
    print("/datalab/nuclei_seg/4_class/train/border dir has been made")
if not os.path.exists('/datalab/nuclei_seg/4_class/train/background'):    
    os.mkdir('/datalab/nuclei_seg/4_class/train/background')
    print("/datalab/nuclei_seg/4_class/train/background dir has been made")

if not os.path.exists('/datalab/nuclei_seg/4_class/train/inter'):    
    os.mkdir('/datalab/nuclei_seg/4_class/train/inter')
    print("/datalab/nuclei_seg/4_class/train/inter dir has been made")

    
for im_num_train,im_name_train in enumerate(images_name_train):
    im_name_train=os.path.join('/datalab/nuclei_seg/norm_ideal',im_name_train)
    binary_img_name=im_name_train.replace('norm_ideal','4_class/binary_images')
    binary_img_train=cv2.imread(binary_img_name,0)

    
    normal_img=cv2.imread(im_name_train,1) ##CHECK
    normal_img=pad_image(normal_img)
    file_name=pattern.search(im_name_train)
    file_name=file_name.group(1)
    nucleus_indices=np.where(binary_img_train==127)
    border_indices=np.where(binary_img_train==255)
    inter_indices=np.where(binary_img_train==63)
    bg_indices=np.where(binary_img_train==0)
        
    
    indices=[nucleus_indices,inter_indices,border_indices,bg_indices]
    directory_name=['nucleus','inter','border','background']

    

    for iter_number,ind_values in enumerate(indices):
        
        
        print("Started extracting indices of {}".format(directory_name[iter_number]))
        X_value,Y_value=ind_values
        ind=np.random.randint(0,len(X_value)-1,4750)
        X_value=X_value[ind]
        Y_value=Y_value[ind]
  
        
        dir_name=directory_name[iter_number]

        for i,x in enumerate(X_value):
            y=Y_value[i]
            if x<=25 or y<=25:
                continue
            y=y+25
            x=x+25
            sub_image=normal_img[x-25:x+26,y-25:y+26,:]
            sub_image_name='/datalab/nuclei_seg/4_class/train/{}/{}_{}.PNG'.format(dir_name,file_name,i)
            cv2.imwrite(sub_image_name,sub_image)
            print("Sub image {}/{} of file {},shaped {} is done".format(i+1,len(X_value),im_num_train+1,sub_image.shape))

            
print("DONE")

In [ ]:
minimum_class=np.min([len(os.listdir('/datalab/nuclei_seg/4_class/train/nucleus'))\
                      ,len(os.listdir('/datalab/nuclei_seg/4_class/train/border'))\
                      ,len(os.listdir('/datalab/nuclei_seg/4_class/train/background'))])
dir_name_train='/datalab/nuclei_seg/4_class/train'
train_sub_dir=os.listdir('/datalab/nuclei_seg/4_class/train')
for directory in tqdm(train_sub_dir):
    directory=os.path.join(dir_name_train,directory)
    if len(os.listdir(directory))>minimum_class:
        dele_files=np.random.choice(os.listdir(directory),len(os.listdir(directory))-minimum_class)
        for files in dele_files:
            files=os.path.join(directory,files)
            try:
                os.remove(files)
            except OSError:
                pass

            

        

In [ ]:
print(len(os.listdir('/datalab/nuclei_seg/4_class/train/nucleus'))\
                      ,len(os.listdir('/datalab/nuclei_seg/4_class/train/border'))\
                      ,len(os.listdir('/datalab/nuclei_seg/4_class/train/background'))\
     ,len(os.listdir('/datalab/nuclei_seg/4_class/train/inter')))

In [ ]:

pattern=re.compile('/datalab/nuclei_seg/norm_ideal/(.*).PNG')
#norm_images_name=glob.glob('/datalab/nuclei_seg/norm_ideal/*.PNG')
test_set=['TCGA-E2-A1B5-01Z-00-DX1.PNG','TCGA-E2-A14V-01Z-00-DX1.PNG',\
                  'TCGA-B0-5698-01Z-00-DX1.PNG','TCGA-B0-5710-01Z-00-DX1.PNG',\
                  'TCGA-21-5786-01Z-00-DX1.PNG','TCGA-21-5784-01Z-00-DX1.PNG',\
                  'TCGA-G9-6356-01Z-00-DX1.PNG','TCGA-G9-6362-01Z-00-DX1.PNG']
images_name_test=random.sample(test_set,4)


if not os.path.exists('/datalab/nuclei_seg/4_class/test'):
    os.mkdir('/datalab/nuclei_seg/4_class/test')
    print(' test directory has been made')
if not os.path.exists('/datalab/nuclei_seg/4_class/test/nucleus'):
    os.mkdir('/datalab/nuclei_seg/4_class/test/nucleus')
    print("/datalab/nuclei_seg/4_class/test/nucleus dir has been made")
if not os.path.exists('/datalab/nuclei_seg/4_class/test/border'):
    os.mkdir('/datalab/nuclei_seg/4_class/test/border')
    print("/datalab/nuclei_seg/4_class/test/border dir has been made")
if not os.path.exists('/datalab/nuclei_seg/4_class/test/background'):    
    os.mkdir('/datalab/nuclei_seg/4_class/test/background')
    print("/datalab/nuclei_seg/4_class/test/background dir has been made")

if not os.path.exists('/datalab/nuclei_seg/4_class/test/inter'):    
    os.mkdir('/datalab/nuclei_seg/4_class/test/inter')
    print("/datalab/nuclei_seg/4_class/test/inter dir has been made")

    
for im_num_test,im_name_test in enumerate(images_name_test):
    im_name_test=os.path.join('/datalab/nuclei_seg/norm_ideal',im_name_test)
    binary_img_name=im_name_test.replace('norm_ideal','4_class/binary_images')
    binary_img_test=cv2.imread(binary_img_name,0)

    
    normal_img=cv2.imread(im_name_test,1) ##CHECK
    normal_img=pad_image(normal_img)
    file_name=pattern.search(im_name_test)
    file_name=file_name.group(1)
    nucleus_indices=np.where(binary_img_test==127)
    border_indices=np.where(binary_img_test==255)
    inter_indices=np.where(binary_img_test==63)
    bg_indices=np.where(binary_img_test==0)
        
    
    indices=[nucleus_indices,inter_indices,border_indices,bg_indices]
    directory_name=['nucleus','inter','border','background']

    

    for iter_number,ind_values in enumerate(indices):
        
        
        print("Started extracting indices of {}".format(directory_name[iter_number]))
        X_value,Y_value=ind_values
        ind=np.random.randint(0,len(X_value)-1,4750)
        X_value=X_value[ind]
        Y_value=Y_value[ind]
  
        
        dir_name=directory_name[iter_number]

        for i,x in enumerate(X_value):
            y=Y_value[i]
            if x<=25 or y<=25:
                continue
            y=y+25
            x=x+25
            sub_image=normal_img[x-25:x+26,y-25:y+26,:]
            sub_image_name='/datalab/nuclei_seg/4_class/test/{}/{}_{}.PNG'.format(dir_name,file_name,i)
            cv2.imwrite(sub_image_name,sub_image)
            print("Sub image {}/{} of file {},shaped {} is done".format(i+1,len(X_value),im_num_test+1,sub_image.shape))

            
print("DONE")


In [ ]:
x_train=[]
for  direc in tqdm(os.listdir('/datalab/nuclei_seg/4_class/train')):
    train_images=glob.glob('/datalab/nuclei_seg/4_calss/train/{}/*.PNG'.format(direc))
    
    for img in (train_images):
        temp=cv2.imread(img,1)
        x_train.append(temp)



In [6]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,PATH,samplewise_center=True, samplewise_std_normalization=True\
                 ,batch_size=256,shuffle=True):
        self.PATH=PATH
        self.encoding_style={}
        self.no_cat=len(os.listdir(self.PATH))
        print("Number of classes : {}".format(self.no_cat))
        self.sum_image=0
        for i,category in enumerate(os.listdir(self.PATH)):
            self.encoding_style[category]=i
            self.sum_image+=len(os.listdir(os.path.join(self.PATH,category)))
        print("Total number of images : {}".format(self.sum_image) )
        print("INDICES : {}".format(self.encoding_style))
    
        
        self.samplewise_center=samplewise_center
        self.samplewise_std_normalization=samplewise_std_normalization
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(self.sum_image/ self.batch_size))

    def __getitem__(self, index):
    
        batch_image_path= self.image_paths[index*self.batch_size:(index+1)*self.batch_size]
        X=[]
        y=[]
        for batch_element_path in batch_image_path:
    
            X_temp, y_temp = self.__data_generation(batch_element_path)
            X.append(X_temp)
            y.append(y_temp)
        
        X=np.array(X)
        y=np.array(y)

        return X, y
    
    
    def on_epoch_end(self):
        
        self.image_paths = []
        classes_path=[self.PATH+'/'+x for x in os.listdir(self.PATH)]
        
        for cls in (classes_path):
            for image_name in os.listdir(cls):
                
                image_path=os.path.join(cls,image_name)
                self.image_paths.append(image_path)
        
        if self.shuffle == True:
                np.random.shuffle(self.image_paths)
        

    def __data_generation(self, batch_element_path):
        class_name=batch_element_path.split('/')[-2]
        class_index=self.encoding_style[class_name]
    
        X = imread(batch_element_path)
        if self.samplewise_center==True:
            X[:,:,0]=(X[:,:,0]-np.mean(X[:,:,0]))/np.std(X[:,:,0])
            X[:,:,1]=(X[:,:,1]-np.mean(X[:,:,1]))/np.std(X[:,:,1])
            X[:,:,2]=(X[:,:,2]-np.mean(X[:,:,2]))/np.std(X[:,:,2])
        if self.samplewise_std_normalization==True:
            X[:,:,0]=(X[:,:,0])/np.std(X[:,:,0])
            X[:,:,1]=(X[:,:,1])/np.std(X[:,:,1])
            X[:,:,2]=(X[:,:,2])/np.std(X[:,:,2])
        #X=np.expand_dims(X,axis=0)
        y=to_categorical(class_index,self.no_cat)


        return X,y

In [8]:
os.chdir("/datalab/nuclei_seg")
print("PWD : {}".format(os.getcwd()))
TRAIN_PATH='/datalab/nuclei_seg/4_class/train'
TEST_PATH='/datalab/nuclei_seg/4_class/test'

#datagen=image.ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
#datagen.fit(x_train)
'''
train_generator=datagen.flow_from_directory(directory=TRAIN_PATH,target_size=(51,51),\
                                              color_mode='rgb',batch_size=256,class_mode='categorical',\
                                              shuffle=True,seed=None)
test_generator=datagen.flow_from_directory(directory=TEST_PATH,target_size=(51,51),\
                                             color_mode='rgb',batch_size=256,class_mode='categorical',\
                                          shuffle=True,seed=None)

'''
train_generator=DataGenerator(TRAIN_PATH)
test_generator=DataGenerator(TEST_PATH)

PWD : /datalab/nuclei_seg
Number of classes : 4
Total number of images : 215288
INDICES : {'border': 0, 'nucleus': 1, 'background': 2, 'inter': 3}
Number of classes : 4
Total number of images : 72739
INDICES : {'nucleus': 0, 'background': 1, 'inter': 2, 'border': 3}


In [9]:
class ReduceLR(keras.callbacks.Callback):
    
    def __init__(self, monitor='val_loss', factor=0.1, patience=10,
                 verbose=0, cooldown=0, min_lr=0):

        self.monitor = monitor
        if factor >= 1.0:
            raise ValueError('ReduceLROnPlateau '
                             'does not support a factor >= 1.0.')
        self.factor = factor
        self.min_lr = min_lr
        self.patience = patience
        self.verbose = verbose
        self.cooldown = cooldown
        self.cooldown_counter = 0  # Cooldown counter.
        self.wait = 0
        self.prev = 0
        self.monitor_op = None
        self._reset()

    def _reset(self):
        
        self.monitor_op = lambda a, b: a!=b
        self.prev = 0
        self.cooldown_counter = 0
        self.wait = 0
       

    def on_train_begin(self, logs=None):
        self._reset()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        current = logs.get(self.monitor)
        print("Wait : {}".format(self.wait))
        if self.wait==0:
            self.prev=current
       
        if self.in_cooldown():
            self.cooldown_counter -= 1
            self.wait = 0

        elif not self.in_cooldown():
            if self.monitor_op(current,self.prev):
                self.wait=0
            else:
                self.wait += 1
                
            if self.wait >= self.patience:
                old_lr = float(K.get_value(self.model.optimizer.lr))
                if old_lr > self.min_lr:
                    new_lr = old_lr * self.factor
                    new_lr = max(new_lr, self.min_lr)
                    K.set_value(self.model.optimizer.lr, new_lr)
                    if self.verbose > 0:
                        print('\nEpoch {}: ReduceLROnPlateau reducing '
                              'learning rate to {}.'.format(epoch + 1, new_lr))
                    self.cooldown_counter = self.cooldown
                    self.wait = 0

    def in_cooldown(self):
        return self.cooldown_counter > 0


In [10]:
def f1_score(y_true,y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    return K.mean(f1)

class CheckWeights(tf.keras.callbacks.Callback):
    def __init__(self):
        self.weights_dif=[]
    
    def on_epoch_end(self,epoch,logs):
        print("LR : {}".format(K.eval(self.model.optimizer.lr)))
        weights,_biases=model.layers[-1].get_weights()
        self.weights_dif.append(weights)
        if len(self.weights_dif)==2:
            l1=np.subtract(self.weights_dif[1],self.weights_dif[0])
            print("l1 norm = {},{}".format(np.max(l1),l1))
            self.weights_dif=[]
            
        
class StoreWeights(tf.keras.callbacks.Callback):
    def __init__(self,file_path,epoch_check):
        self.file_path=file_path
        self.epoch_check=epoch_check
    def on_epoch_end(self,epoch,logs=None):
        if epoch==self.epoch_check:
            self.model.save_weights(self.file_path, overwrite=True)
            
        

In [11]:
class CosineWithRestart(keras.callbacks.Callback):
    
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=100,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            if self.cycle_length>= self.steps_per_epoch:
                self.cycle_length = self.steps_per_epoch
                
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)
        


In [14]:
def expend_as(tensor, rep):
        my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep})(tensor)
        return my_repeat

def AttnGatingBlock(x, g, inter_shape):
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same')(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same')(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    # my_repeat=Lambda(lambda xinput:K.repeat_elements(xinput[0],shape_x[1],axis=1))
    # upsample_psi=my_repeat([upsample_psi])
    upsample_psi = expend_as(upsample_psi, shape_x[3])

    y = multiply([upsample_psi, x])

    # print(K.is_keras_tensor(upsample_psi))

    result = Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = BatchNormalization()(result)
    return result_bn

def UnetGatingSignal(input, is_batchnorm=True):
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 2, (1, 1), strides=(1, 1), padding="same")(input)
    if is_batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def attn_unet(pretrained_weights = None,input_size = (512,512,3),act='relu',optimizer='adam'):
    inputs = Input(input_size)
    assert act in ['relu','leaky_relu']
    assert optimizer in ['adam','sgd']
    if act=='relu':
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        #bach_size x 512 x 512 x 64
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
        #bach_size x 512 x 512 x 64
    else:
        conv1 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1=LeakyReLU(alpha=0.5)(conv1)
        #bach_size x 512 x 512 x 64
        conv1 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv1=LeakyReLU(alpha=0.5)(conv1)
        #bach_size x 512 x 512 x 64
    activation='relu'
  
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #bach_size x 256 x 256 x 64
    conv2 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool1)
    #bach_size x 256 x 256 x 128
    conv2 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    #bach_size x 256 x 256 x 128
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #bach_size x 128 x 128 x 128
    conv3 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool2)
    #bach_size x 128 x 128 x 256
    conv3 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv3)
    #bach_size x 128 x 128 x 256
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #bach_size x 64 x 64 x 256
    conv4 = Conv2D(512, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool3)
    #bach_size x 64 x 64 x 512
    conv4 = Conv2D(512, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv4)
    #bach_size x 64 x 64 x 512
    drop4 = Dropout(0.43)(conv4)
    
    #pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    #conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    #conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    #drop5 = Dropout(0.5)(conv5)

    #up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    #merge6 = concatenate([drop4,up6], axis = 3)
    #conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    #conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    
    
    g1=UnetGatingSignal(drop4)
    #bach_size x 64 x 64 x 1024
    att1=AttnGatingBlock(conv3, g1, 512)
    #bach_size x 128 x 128 x 256
    up7 = Conv2D(256, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop4))#(conv6))
    #bach_size x 128 x 128 x 256
    merge7 = concatenate([att1,up7], axis = 3)
    #bach_size x 128 x 128 x 512
    conv7 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge7)
    #bach_size x 128 x 128 x 256
    conv7 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv7)
    #bach_size x 128 x 128 x 256
    
    g2=UnetGatingSignal(conv7)
    #bach_size x 128 x 128 x 512
    att2=AttnGatingBlock(conv2, g2, 256)
    #bach_size x 256 x 256 x 128

    up8 = Conv2D(128, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    #bach_size x 256 x 256 x 128
    merge8 = concatenate([att2,up8], axis = 3)
    #bach_size x 256 x 256 x 256
    conv8 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge8)
    #bach_size x 256 x 256 x 128
    conv8 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv8)
    #bach_size x 256 x 256 x 128
    
    g3=UnetGatingSignal(conv8)
    #bach_size x 256 x 256 x 256
    att3=AttnGatingBlock(conv1, g3, 128)
    #bach_size x 512 x 512 x 64

    up9 = Conv2D(64, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    #bach_size x 512 x 512 x 64
    merge9 = concatenate([att3,up9], axis = 3)
     #bach_size x 512 x 512 x 128
    conv9 = Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge9)
     #bach_size x 512 x 512 x 64
    conv9 = Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
     #bach_size x 512 x 512 x 64
    conv9 = Conv2D(2, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
     #bach_size x 512 x 512 x 2
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
     #bach_size x 512 x 512 x 1

    model = Model(inputs = inputs, outputs = conv10)
    
    if optimizer=='adam':
        OPT=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0000, amsgrad=False)
        print('adam optimizer')
    elif optimizer=='sgd':
        OPT=keras.optimizers.SGD(lr=0.01,momentum=0.8,nesterov=True)
        print('SGD optimizer')
    model.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef,recall,auc])
    

    return model



model already exists in model_4class_2019_08_14
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 51, 51, 3)         0         
_________________________________________________________________
conv_1_layer (Conv2D)        (None, 48, 48, 25)        1225      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 25)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 25)        0         
_________________________________________________________________
conv_2_layer (Conv2D)        (None, 20, 20, 50)        31300     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 50)        0         
_________________________________________________________________
dropout_7 (Dropout)         

W0814 16:23:51.600939 140399403984704 deprecation.py:323] From /opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/115
 71/841 [=>............................] - ETA: 1:01:23 - loss: 1.3865 - acc: 0.2485 - f1_score: 0.0000e+00

Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
dir_content=os.listdir()
model_list=[s for s in dir_content if 'model_' in s]
model_list=sorted(model_list)
model_folder=os.path.join(os.getcwd(),model_list[-2])
print("available models are :",model_list,"\n","selected model is :", model_folder)
model_weights=os.path.join(os.getcwd(),model_folder)+"/"+[weights for weights in os.listdir(model_folder) if 'model_final' in weights][-1]
model_architecture=glob.glob(os.path.join(os.getcwd(),model_folder)+"/*.json")[-1]
print(model_weights,model_architecture)




In [ ]:
json_file = open(model_architecture, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
OPT=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

loaded_model.load_weights(model_weights)
loaded_model.compile(loss='categorical_crossentropy',metrics=['accuracy',f1_score],optimizer=OPT)
print(loaded_model.metrics_names)

In [ ]:
def make_predictions(img,model):
    padded_img=pad_image(img)
    pred_nucleus=[]
    pred_border=[]
    pred_bg=[]
    for X in range(img.shape[0]):
        x=X+25
        sub_image=[]
        for Y in range(img.shape[1]):
            y=Y+25
            sub_image.append(padded_img[x-25:x+26,y-25:y+26,:])
        sub_image=np.array(sub_image)
        pred=(model.predict(sub_image,batch_size=32,verbose=1))
        pred_nucleus.append(pred[:,2])
        pred_border.append(pred[:,1])
        pred_bg.append(pred[:,0])
        
    pred_nucleus=np.array(pred_nucleus)
    pred_border=np.array(pred_border)
    pred_bg=np.array(pred_bg)
    
    return pred_nucleus,pred_border,pred_bg

In [ ]:
print(trai,directory)

In [ ]:
img_path="/atheeth/nuclei_seg/norm_ideal/TCGA-18-5592-01Z-00-DX1.PNG"
img_pred1=cv2.imread(img_path,1)
img_pred=img_pred1/255

In [ ]:
!pwd

In [ ]:
nucleus,border,bg=make_predictions(img_pred,model)
n=nucleus.copy()
n[n<0.5]=0
n[n>=0.5]=1

In [ ]:
import scipy
boundry=border
labeled_nucleus,_=scipy.ndimage.measurements.label(n)
image_binary=np.zeros(img_pred.shape[0:1])
image_color=np.zeros(img_pred.shape)
dilation_filter=np.array([[0,1,0],[1,1,1],[0,1,0]],np.uint8)
boundry_mean=np.mean(boundry[boundry!=0])
max_label=np.amax(labeled_nucleus)
current=np.zeros((max_label))


from tqdm import tqdm
for i in tqdm(range(max_label)):
    #print("Region {}/{}".format(i+1,max_label))
    temp_binary=np.zeros((1000,1000),np.uint8)
    temp_binary[labeled_nucleus==i]=np.uint8(1)
    temp_color=np.zeros((1000,1000,3))
    temp_color[:,:,0]=np.random.randint(0,100)*temp_binary
    temp_color[:,:,1]=np.random.randint(0,100)*temp_binary
    temp_color[:,:,2]=np.random.randint(0,100)*temp_binary
    temp_color=temp_color.astype(np.uint8)
    prev=0
    count=0
    while current[i]<boundry_mean and count<2:
        prev=current[i]
        temp1=cv2.dilate(n,dilation_filter,iterations = 1)
        temp2=cv2.dilate(temp_color,dilation_filter)
        count+=1
        diff=(temp1-temp_binary).astype(np.uint8)
        temp_multi=np.matmul(diff,boundry)
        current[i]=np.mean(temp_multi[temp_multi!=0])
    image_binary=image_binary+temp_binary
    image_color=image_color+temp_color

        
    


In [ ]:
print(model.predict(np.array([img_pred[51:102,51:102]])))
#print(loaded_model.predict(np.array([img_pred[51:102,51:102]])))

In [ ]:
list_dir=os.listdir(os.path.join(os.getcwd(),'train/background'))
print(len(list_dir))